In [2]:
import torch
from torch import nn
from torch.nn import functional as F
import math

In [ ]:
# 伪代码
class MutiHeadAttention(nn.Module):
    def __init__(self,*args):
        super().__init__()

In [ ]:
# 伪代码
class LayerNorm(nn.Module):
    def __init__(self,*args):
        super().__init__()

In [ ]:
# 伪代码
class PositionWiseFeedForward(nn.Module):
    def __init__(self,*args):
        super().__init__()

* 输入向量：$\mathbf{x}$
* Encoder 输出（记作 memory）：$\mathbf{M}$
* 各个子层的结果：依次是 $\mathbf{x}_1, \mathbf{x}_2, \mathbf{y}$

## DecoderLayer 的公式

### 1. Masked Multi-Head Self-Attention

对目标序列的输入 $\mathbf{x}$，做带掩码的多头注意力：

$$
\mathbf{z}_1 = \text{MultiHeadAttention}(\mathbf{x}, \mathbf{x}, \mathbf{x}, \text{mask})
$$

加上残差 & LayerNorm：

$$
\mathbf{x}_1 = \text{LayerNorm}(\mathbf{x} + \mathbf{z}_1)
$$

---

### 2. Encoder–Decoder Attention (Cross-Attention)

以 $\mathbf{x}_1$ 作为 Query，Encoder 的输出 $\mathbf{M}$ 作为 Key 和 Value：

$$
\mathbf{z}_2 = \text{MultiHeadAttention}(\mathbf{x}_1, \mathbf{M}, \mathbf{M})
$$

加上残差 & LayerNorm：

$$
\mathbf{x}_2 = \text{LayerNorm}(\mathbf{x}_1 + \mathbf{z}_2)
$$

---

### 3. Position-wise Feed Forward Network (FFN)

FFN 通常是两层前馈网络：

$$
\text{FFN}(\mathbf{x}_2) = \max(0, \mathbf{x}_2 \mathbf{W}_1 + \mathbf{b}_1)\mathbf{W}_2 + \mathbf{b}_2
$$

加上残差 & LayerNorm：

$$
\mathbf{y} = \text{LayerNorm}(\mathbf{x}_2 + \text{FFN}(\mathbf{x}_2))
$$

---

## Multi-Head Attention 的细节公式

每个 **注意力头（head）**：

$$
\text{Attention}(Q, K, V) = \text{softmax}\!\left(\frac{QK^\top}{\sqrt{d_k}} + \text{mask}\right)V
$$

多头拼接后：

$$
\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1, \dots, \text{head}_h)\mathbf{W}^O
$$

其中：

$$
\text{head}_i = \text{Attention}(Q\mathbf{W}^Q_i, K\mathbf{W}^K_i, V\mathbf{W}^V_i)
$$

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,d_model,ffn_hidden,drop_prod):
        super().__init__()
        self.d_model=d_model
        self.ffn_hidden=ffn_hidden
        self.drop_prod=drop_prod
        self.attention=MutiHeadAttention(self.d_model,self.n_head)
        self.norm=LayerNorm(self.d_model)
        self.dropout=nn.Dropout(self.drop_prod)
        self.ffn=PositionWiseFeedForward(self.d_model,self.ffn_hidden,self.drop_prod)
    def forward(self,dec,enc,t_mask,s_mask):
